# Libs

In [88]:
import time
import json

import pandas as pd
import requests
from bs4 import BeautifulSoup as BS

# Wunderground

In [3]:
url = "http://api.wunderground.com/api/2b0d6572c90d3e4a/lang:CN/conditions_v11/alerts_v11/forecast/forecast10day_v11/hourly10day_v11/history_2018101820181028/astronomy/astronomy10day/q/zmw:00000.1.58362.json?v=wuiapp"
res = requests.get(url)
res.encoding = 'utf-8'
datadict = json.loads(res.text)

In [196]:
data = pd.DataFrame(datadict['history']['observations'])


def concat_datetime(x):
    Y, m, d = x['year'], x['mon'], x['mday']
    H, M = x['hour'], x['min']

    datetime = ''
    for item in [Y, m, d]:
        datetime = datetime + '/' + item
    datetime += (' ' + H)
    datetime += (':' + M)

    return datetime[1:]


data['datetime'] = data['date'].map(lambda x: concat_datetime(x))
data['datetime'] = pd.to_datetime(data['datetime'])

dropcols = ['date', 'utcdate', 'dewpti', 'dewptm', 'hail', 'tempi', 'heatindexi',
            'heatindexm', 'icon', 'metar', 'precipi', 'precipm', 'pressurei',
            'pressurem', 'snow', 'tempi', 'thunder', 'tornado', 'visi', 'vism',
            'wdird', 'wdire', 'wgusti', 'wgustm', 'windchilli', 'windchillm', 'wspdi']

colnames = ['skyCondition', 'fog', 'humidity', 'rain',
            'temperature', 'windSpeed', 'datetime']

data.drop(dropcols, axis=1, inplace=True)
data.columns = colnames

In [197]:
data.to_csv('../Data/MeteroData.csv',index=None)

# Baidu Index

In [73]:
url_congest_mile = "https://jiaotong.baidu.com/trafficindex/city/congestmile?cityCode=289"
url_roadrank0 = "https://jiaotong.baidu.com/trafficindex/city/roadrank?cityCode=289&roadtype=0"
url_roadrank1 = "https://jiaotong.baidu.com/trafficindex/city/roadrank?cityCode=289&roadtype=1"
url_cities = "https://jiaotong.baidu.com/trafficindex/city/road?cityCode=289"

In [66]:
def get_json(url):
    res = requests.get(url)
    res.encoding = 'utf-8'
    return json.loads(res.text)

In [93]:
def get_roadrank_data(url_roadrank, roadRankData):
    for url in url_roadrank:
        raw_df = pd.DataFrame(get_json(url)['data']['list'])
        raw_df.drop(['citycode','district_type','links',
                     'nameadd','roadsegid','semantic'],
                    axis = 1,inplace = True)
        raw_df.columns = roadRankData.columns
        roadRankData = pd.concat((roadRankData,raw_df),axis=0)

    return roadRankData

In [91]:
def update_and_save(df, fn, url, name):
    df_update = fn(url, df)
    df_update.to_csv('./' + name + '.csv',index = None)
    time.sleep(300)

In [ ]:
roadRankData = pd.DataFrame(columns=['id','index','indexLevel','length',
                                     'location','roadType','roadName',
                                     'speed','datetime','congestionLength'])
update_and_save(roadRankData, get_roadrank_data,
                url = [url_roadrank0,url_roadrank1],
                name = 'RoadRankData')